In [1]:
'''
In order to properly setup my local environment:
- pip install boto3
- I had to install Spark, Java, Hadoop, and SBT on my Windows 10 machine, ADD YT LINK FOR SBT TOO!!
following the instructions at this link: https://www.youtube.com/watch?v=g7Qpnmi0Q-s for the Java/Hadoop portion.
'''

'\nIn order to properly setup my local environment:\n- pip install boto3\n- I had to install Spark, Java, Hadoop, and SBT on my Windows 10 machine, ADD YT LINK FOR SBT TOO!!\nfollowing the instructions at this link: https://www.youtube.com/watch?v=g7Qpnmi0Q-s for the Java/Hadoop portion.\n'

In [2]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import types as T
import  pyspark.sql.functions as F
import time

In [3]:
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

In [4]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [5]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [6]:
spark = create_spark_session()
spark

In [ ]:
# use this to speed up parquet write
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("mapreduce.fileoutputcommitter.algorithm.version", "2")

In [7]:
sc._jsc.hadoopConfiguration().set("spark.speculation","false")

NameError: name 'sc' is not defined

In [12]:
# Not sure that second line is faster than first
#df = spark.read.format("json").load("s3a://udacity-dend/song_data/*/*/*") # runs in 13 minutes
#df = spark.read.format("json").load(file_locations)
# df = spark.read.format("json").load(songs_paths)
# song_data = input_data+'song_data/*/*/*/*.json
# song_data = input_data + 'song_data/*/*/*/*.json # runs in 14 minutes
# df = spark.read.json(song_data) # runs in 14 minutes
# df = spark.read.json(songs_paths) # runs in 26 minutes

start = time.time()
#input_data = "s3a://udacity-dend/"
#song_data =  input_data + 'song_data/*/*/*/*.json'
df = spark.read.json("s3a://udacity-dend/song_data/*/*/*") # runs in 13 minutes
end = time.time()
print('runtime (s):', end-start)

runtime (s): 529.4650149345398


In [11]:
'''
start = time.time()
print(type(df))
print(df)
print('count', df.count())
#df.printSchema()
end = time.time()
print('runtime (s):', end-start)
'''

<class 'pyspark.sql.dataframe.DataFrame'>
DataFrame[artist_id: string, artist_latitude: double, artist_location: string, artist_longitude: double, artist_name: string, duration: double, num_songs: bigint, song_id: string, title: string, year: bigint]
count 14896
runtime (s): 339.83151745796204


In [83]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://dend-sparkify-amiri/"

In [84]:
# extract columns to create songs table
start = time.time()
songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()
end = time.time()
print('extract columns to create songs table runtime (s):', end-start)

# write songs table to parquet files partitioned by year and artist
start = time.time()
songs_table_parquet = df.select("song_id", "title", "artist_id", "year", "duration", col("artist_name").alias("artist")).distinct()
#songs_table_parquet.write.mode('overwrite').partitionBy("year", "artist").parquet(output_data + "songs/") # S3 location
songs_table_parquet.write.mode('overwrite').partitionBy("year", "artist").parquet('output_data/' + "songs/") # local location
end = time.time()
print('write songs table to parquet files runtime (s):', end-start)

AnalysisException: cannot resolve '`song_id`' given input columns: [artist, auth, firstName, gender, itemInSession, lastName, length, level, location, method, page, registration, sessionId, song, start_time, status, ts, userAgent, userId];;
'Project ['song_id, 'title, 'artist_id, 'year, 'duration]
+- Project [artist#136, auth#137, firstName#138, gender#139, itemInSession#140L, lastName#141, length#142, level#143, location#144, method#145, page#146, registration#147, sessionId#148L, song#149, status#150L, ts#151L, userAgent#152, userId#153, <lambda>(ts#151L) AS start_time#189]
   +- Filter (page#146 = NextSong)
      +- Relation[artist#136,auth#137,firstName#138,gender#139,itemInSession#140L,lastName#141,length#142,level#143,location#144,method#145,page#146,registration#147,sessionId#148L,song#149,status#150L,ts#151L,userAgent#152,userId#153] json


In [16]:
# extract columns to create artists table
start = time.time()
artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()
end = time.time()
print('extract columns to create artists table runtime (s):', end-start)

# write artists table to parquet files
start = time.time()
artists_table_parquet = artists_table.write.mode('overwrite').parquet(output_data + "artists/")
end = time.time()
print('write artists table to parquet files runtime (s):', end-start)

extract columns to create artists table runtime (s): 0.09255576133728027
write artists table to parquet files runtime (s): 444.856107711792


## Put finalized code from above in sections below, then transfer to .py file. 

In [ ]:
def process_song_data(spark, input_data, output_data):
    '''
    This function reads the data from S3, processes the song data using Spark, and writes the processed data back to S3.

    Parameters:
    - spark: The spark session
    - input_data: The S3 path location up to, but not including `song_data`
    - output_data: The S3 bucket where the new dimensional tables will be written to
    '''
    # get filepath to song data file
    start = time.time()
    song_data =  input_data + 'song_data/*/*/*/*.json'
    end = time.time()
    print('runtime (s):', end-start)
    
    # read song data file
    start = time.time()
    df = spark.read.json(song_data) # may not need this new schema
    end = time.time()
    print('read song data file runtime (s):', end-start)

    # extract columns to create songs table
    start = time.time()
    songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()
    end = time.time()
    print('extract columns to create songs table runtime (s):', end-start)

    # write songs table to parquet files partitioned by year and artist
    start = time.time()
    songs_table_parquet = df.select("song_id", "title", "artist_id", "year", "duration", col("artist_name").alias("artist")).distinct()
    songs_table_parquet.write.mode('overwrite').partitionBy("year", "artist_id").parquet(output_data + "songs/")
    end = time.time()
    print('write songs table to parquet files runtime (s):', end-start)

    # extract columns to create artists table
    start = time.time()
    artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()
    end = time.time()
    print('extract columns to create artists table runtime (s):', end-start)

    # write artists table to parquet files
    start = time.time()
    artists_table_parquet = artists_table.write.mode('overwrite').parquet(output_data + "artists/")
    end = time.time()
    print('write artists table to parquet files runtime (s):', end-start)

In [ ]:
def process_log_data(spark, input_data, output_data):
   '''
    This function reads the data from S3, processes the log data using Spark,
    and writes the processed data back to S3.
   '''
    # get filepath to log data file
    start = time.time()
    log_data = input_data + 'log_data/*/*/*.json'
    end = time.time()
    print('get log filepath runtime (s):', end-start)

    # read log data file
    start = time.time()
    df = spark.read.json(log_data)
    end = time.time()
    print('read log data file runtime (s):', end-start)

    # filter by actions for song plays
    df = df.filter(df.page=='NextSong')

    # extract columns for users table    
    start = time.time()
    users_table = df.select("user_id", "first_name", "last_name", "gender", "level").distinct()
    end = time.time()
    print('extract columns for users table runtime (s):', end-start)
    
    # write users table to parquet files
    start = time.time()
    users_table.write.mode('overwrite').partitionBy("year", "artist_id").parquet(output_data + "users/")
    end = time.time()
    print('write users table to parquet files runtime (s):', end-start)

    # create timestamp column from original timestamp column
    get_timestamp = F.udf(lambda x: datetime.fromtimestamp(x/1000.0), T.TimestampType() )
    df = df.withColumn("timestamp", get_timestamp(df.ts)) # creates new column named timestamp, populates it with converted timestamp
    
    # create datetime column from original timestamp column
    get_datetime = F.udf(lambda x: datetime.fromtimestamp(x/1000.0) )
    df = df.withColumn("datetime", get_timestamp(df.ts))
    
    # extract columns to create time table
    time_table = df.select(datetime)
    time_table = time_table.withColumn("month", F.month("datetime"), )
    
    # write time table to parquet files partitioned by year and month
    time_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "time/")

    # read in song data to use for songplays table
    song_df = 

    # extract columns from joined song and log datasets to create songplays table 
    songplays_table = 

    # write songplays table to parquet files partitioned by year and month
    songplays_table

In [ ]:
def main():
    '''
    Create Spark session, provide paths to input/output data, load songs/log data
    and create parquet tables (columnar format) with star schema DB.
    '''
    spark = create_spark_session()
    input_data = "s3a://udacity-dend/"
    output_data = "s3a://dend-sparkify-amiri/output_data"
    
    process_song_data(spark, input_data, output_data)    
    process_log_data(spark, input_data, output_data)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# SANDBOX AREA BELOW ############

In [45]:
# get filepath to log data file
start = time.time()
log_data = input_data + 'log_data/*/*/*.json'
end = time.time()
print('get log filepath runtime (s):', end-start)

get log filepath runtime (s): 0.0009980201721191406


In [46]:
# read log data file
start = time.time()
df = spark.read.json(log_data)
end = time.time()
print('read log data file runtime (s):', end-start)

read log data file runtime (s): 10.974141597747803


In [47]:
df

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [48]:
# filter by actions for song plays
df = df.filter(df.page=='NextSong')

# extract columns for users table    
start = time.time()
users_table = df.select("userid", "firstName", "lastName", "gender", "level").distinct()
end = time.time()
print('extract columns for users table runtime (s):', end-start)

extract columns for users table runtime (s): 0.0638277530670166


In [49]:
# write users table to parquet files
start = time.time()
users_table.write.mode('overwrite').parquet(output_data + "users/")
end = time.time()
print('write users table to parquet files runtime (s):', end-start)

write users table to parquet files runtime (s): 55.40958642959595


In [52]:
'''
# create timestamp column from original timestamp column
get_timestamp = F.udf(lambda x: datetime.fromtimestamp(x/1000.0), T.TimestampType() )
df = df.withColumn("timestamp", get_timestamp(df.ts)) # creates new column named timestamp, populates it with converted timestamp
'''

In [50]:
# create datetime column from original timestamp column
get_datetime = F.udf(lambda x: datetime.fromtimestamp(x/1000.0), T.TimestampType() )
df = df.withColumn("start_time", get_datetime(df.ts))

In [51]:
# extract columns to create time table
time_table = df.select("start_time").distinct().withColumn("hour",    F.hour(df.start_time)) \
                                               .withColumn("day",     F.dayofmonth(df.start_time)) \
                                               .withColumn("week",    F.weekofyear(df.start_time)) \
                                               .withColumn("month",   F.month(df.start_time)) \
                                               .withColumn("year",    F.year(df.start_time)) \
                                               .withColumn("weekday", F.date_format(df.start_time, "E")) # the 'E' formats this parameter to day-of-the-week

In [24]:
time_table

DataFrame[start_time: timestamp, hour: int, day: int, week: int, month: int, year: int, weekday: string]

In [25]:
users_table.write.mode('overwrite').parquet(output_data + "users/")

In [26]:
# write time table to parquet files partitioned by year and month
time_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "time/") 

In [17]:
# read in song data to use for songplays table
song_df_path = 'output_data/' + 'songs/*/*/*'
song_df = spark.read.parquet(song_df_path)

In [31]:
print(type(song_df))
song_df.show()

<class 'pyspark.sql.dataframe.DataFrame'>
+------------------+--------------------+----------+--------------------+
|           song_id|               title|  duration|              artist|
+------------------+--------------------+----------+--------------------+
|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...| 196.04853|Yvonne S. Moriart...|
|SOVPFJK12A6701CB16|Barcelona - (Frie...| 273.44934|Russell Watson / ...|
|SOQEBML12A8C136AA4|Werther (1997 Dig...|  55.40526|Alfredo Kraus/Tat...|
|SODUMDU12AC468A22B|We're Skrewed (Ot...|  249.5473|         All Leather|
|SOPSXLI12A6D4FA418|Practical Cats - ...| 251.03628|Robert Donat/Phil...|
|SOTCIHX12A8C13DDD2|Finally_ as that ...| 483.34322|        Red Sparowes|
|SOKUATC12AB01853F3|Turandot: Nessun ...| 191.13751|Andrea Bocelli / ...|
|SOUFBFK12A8C13D668|String Quartets O...| 348.60363|Endellion String ...|
|SONVWOX12A8C137FF5|Le Roi de Lahore_...| 250.48771|Rolando Villazon/...|
|SOUQKQA12A6D4F951E|Perfume: The Stor...| 331.20608|Berliner Philharm.

In [ ]:
# extract columns from joined song and log datasets to create songplays table 
songplays_table = spark.sql("SELECT s.song_id as songplay_id , l.start_time, l.user_id, l.level, s.song_id, l.artist_id, l.session_id, l.location, l.user_agent  \
                              FROM ( \
                                  SELECT DISTINCT song_df \
                                  FROM song_df \
                              JOIN ( \
                                  )
                                  )
    
    
    
                           ")

In [81]:

song_df.createOrReplaceTempView("song_dataset")
df.createOrReplaceTempView("log_dataset")
song_df2 = spark.sql("SELECT L.start_time, L.userId, L.level, S.song_id, L.artistId, LsessionId, L.location, L.userAgent \
                        FROM log_dataset as L \
                          JOIN song_dataset as S  \
                            ON L.artist = S.artist " )
song_df2.show(5)

AnalysisException: cannot resolve '`L.artistId`' given input columns: [S.artist, L.artist, L.auth, S.duration, L.firstName, L.gender, L.itemInSession, L.lastName, L.length, L.level, L.location, L.method, L.page, L.registration, L.sessionId, L.song, S.song_id, L.start_time, L.status, S.title, L.ts, L.userAgent, L.userId]; line 1 pos 51;
'Project [start_time#189, userId#153, level#143, song_id#16, 'L.artistId, 'LsessionId, location#144, userAgent#152]
+- Join Inner, (artist#136 = artist#19)
   :- SubqueryAlias L
   :  +- SubqueryAlias log_dataset
   :     +- Project [artist#136, auth#137, firstName#138, gender#139, itemInSession#140L, lastName#141, length#142, level#143, location#144, method#145, page#146, registration#147, sessionId#148L, song#149, status#150L, ts#151L, userAgent#152, userId#153, <lambda>(ts#151L) AS start_time#189]
   :        +- Filter (page#146 = NextSong)
   :           +- Relation[artist#136,auth#137,firstName#138,gender#139,itemInSession#140L,lastName#141,length#1

In [75]:
song_df.createOrReplaceTempView("song_dataset")
df.createOrReplaceTempView("log_dataset")
song_df2 = spark.sql("SELECT L.start_time, L.userId, L.level FROM df as L")

AnalysisException: Table or view not found: df; line 1 pos 44;
'Project ['L.start_time, 'L.userId, 'L.level]
+- 'SubqueryAlias L
   +- 'UnresolvedRelation [df]


In [69]:
df

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string, start_time: timestamp]

In [44]:
log_df = spark.read.json(log_data)

NameError: name 'log_data' is not defined

In [55]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
# write songplays table to parquet files partitioned by year and month
songplays_table

In [ ]:
'''
# This code is from the internet, and it works
from pyspark import SparkContext
sc = SparkContext(master = 'local')
create_spark_session()
'''

In [ ]:
# Works! Stops the spark context
SparkSession.stop()

In [ ]:
# !pip install boto3
# Also had to install Java SDK

In [ ]:
# Link to udacity bucket with the console:  https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/?region=us-west-2
#s3path = 'https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/?region=us-west-2#'
#s3pathsong = 'https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/song_data/A/?region=us-west-2'
